In [2]:
import pandas as pd
from datetime import datetime
from dateutil.parser import parse
import re

class DroneAgent:
    def __init__(self, data_dir='data'):
        self.pilots = pd.read_csv(f'{data_dir}/pilot_roster.csv')
        self.drones = pd.read_csv(f'{data_dir}/drone_fleet.csv')
        self.missions = pd.read_csv(f'{data_dir}/missions.csv')

    def save_pilots(self):
        self.pilots.to_csv('data/pilot_roster.csv', index=False)
        print(" Pilot status updated!")

    def save_drones(self):
        self.drones.to_csv('data/drone_fleet.csv', index=False)
        print(" Drone status updated!")

    def query_pilots(self, skill=None, location=None, status='Available'):
        filt = self.pilots['status'] == status
        if skill: filt &= self.pilots['skills'].str.contains(skill, case=False, na=False)
        if location: filt &= self.pilots['location'] == location
        return self.pilots[filt]

    def pilot_cost(self, pilot_id, start_date, end_date):
        pilot = self.pilots[self.pilots['pilotid'] == pilot_id].iloc[0]
        days = (parse(end_date) - parse(start_date)).days + 1
        return int(pilot['dailyrateinr']) * days

    def match_pilots_to_mission(self, project_id):
        mission = self.missions[self.missions['projectid'] == project_id].iloc[0]
        req_skills = [mission['requiredskills']]
        req_certs = mission['requiredcerts'].split(', ')
        loc = mission['location']

        candidates = self.query_pilots(skill=req_skills[0], location=loc)
        matches = []
        for _, p in candidates.iterrows():
            skill_ok = req_skills[0] in p['skills']
            cert_ok = all(c.strip() in p['certifications'] for c in req_certs)
            cost = self.pilot_cost(p['pilotid'], mission['startdate'], mission['enddate'])
            budget_ok = cost <= int(mission['missionbudgetinr'])

            match = {
                'pilot': p['name'], 'cost': cost, 'skills_ok': skill_ok,
                'certs_ok': cert_ok, 'budget_ok': budget_ok, 'loc_match': p['location'] == loc
            }
            matches.append(match)
        return pd.DataFrame(matches)

    def query_drones(self, capability=None, location=None, status='Available', weather=None):
        filt = self.drones['status'] == status
        if capability: filt &= self.drones['capabilities'].str.contains(capability, case=False, na=False)
        if location: filt &= self.drones['location'] == location
        if weather == 'Rainy': filt &= self.drones['weatherresistance'].str.contains('IP43', na=False)
        return self.drones[filt]

    def detect_conflicts(self, assignment=None):
        conflicts = []
        if assignment:
            pilot_id, proj = assignment.split('-')
            p = self.pilots[self.pilots['pilotid'] == pilot_id].iloc[0]
            if p['currentassignment'] != '-': conflicts.append("Double-booked!")
        return conflicts if conflicts else ["No conflicts"]

    def assign_pilot(self, pilot_id, project_id):
        self.pilots.loc[self.pilots['pilotid'] == pilot_id, 'status'] = 'Assigned'
        self.pilots.loc[self.pilots['pilotid'] == pilot_id, 'currentassignment'] = project_id
        self.save_pilots()
        return f" Assigned {pilot_id} to {project_id}"

    def chat(self, query):
        q = query.lower()
        if 'pilot' in q and 'available' in q:
            loc = re.search(r'bangalore|mumbai', q)
            skill = re.search(r'mapping|inspection|survey|thermal', q)
            res = self.query_pilots(skill=skill.group() if skill else None,
                                   location=loc.group() if loc else None)
            return res[['name', 'skills', 'location', 'status']].to_dict('records')

        elif 'match' in q or 'project' in q:
            proj = re.search(r'prj\d+', q.upper()).group() if re.search(r'prj', q) else 'PRJ001'
            return self.match_pilots_to_mission(proj).to_dict('records')

        elif 'drone' in q:
            cap = re.search(r'lidar|rgb|thermal', q)
            res = self.query_drones(capability=cap.group() if cap else None)
            return res[['droneid', 'model', 'status', 'location']].to_dict('records')

        elif 'assign' in q:
            m = re.search(r'P\d+.*?PRJ\d+', q.upper())
            if m:
                pilot, proj = m.group().split()
                return self.assign_pilot(pilot, proj)

        elif 'conflict' in q:
            return self.detect_conflicts()

        elif 'urgent' in q:
            return "Urgent PRJ002: Available pilots in Mumbai with Inspection skills."

        return "Try: 'available pilots Bangalore', 'match PRJ001', 'drones thermal', 'assign P001 PRJ001'"
